In [6]:


import aug_util as aug
import wv_util as wv
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import numpy as np
import csv
import tqdm
import pickle
from sklearn.model_selection import train_test_split
%matplotlib inline


In [3]:
#Loading our labels
_, chips1, classes1 = wv.get_labels('/data/zjc4//xView_train.geojson')

100%|██████████| 601937/601937 [00:03<00:00, 153862.48it/s]


In [166]:
# Grouped classes [1,2,3,4,5]
grouped_classes = [[11,12],[13],[17,18,20,21],\
       [19,23,24,25,28,29,60,61,65,26],[41,42,50,40,44,45,47,49]]
def filterChips(chips,classes):
    # Given train_ind, filter chips,classes
    np.chips
    pass
def transpose_class_counts(chips,classes):
    chip_names = np.unique(chips) 
    results = np.zeros((len(chip_names),5))
    chip_strs = []
    for c_idx, c in tqdm.tqdm(enumerate(chip_names)):
        chip_strs.append(c)
        classes_chip = classes[chips==c]
        idx_filter = np.isin(classes_chip,grouped_classes[0])
        # initialize to all false
        for i,gc in (enumerate(grouped_classes)):
            is_in_idxs = np.isin(classes_chip,gc)
            classes_chip[is_in_idxs] = i
            idx_filter = np.logical_or(idx_filter,is_in_idxs)
        classes_chip = classes_chip[idx_filter]
        labels, counts = np.unique(classes_chip,return_counts=True)
        for label_idx,label in enumerate(labels):
            results[int(c_idx),int(label)] = counts[label_idx]
            pass
    chip_strs_col = np.array(chip_strs).reshape(-1,1)
    return (np.hstack((chip_strs_col,results)))

def indToTifName(data, inds):
    res = []
    for ind in inds:
        res.append(data[ind][0])
    return res

def showDistribution(data, selected_indexes):
    res = []
    total = 0
    class_num = len(data[0])
    for i in range(class_num):
        for index in selected_indexes:
            total += float(data[index][i])
    for i in range(class_num):
        total_of_this_class = 0
        for index in selected_indexes:
            total_of_this_class += float(data[index][i])
        res.append(float(total_of_this_class)/total)
    return res

def checkThreshold(distr1, distr2, thres):
    if (len(distr1) != len(distr2)):
        print("columns' numbers don't fit.")
        return -1
    for i in range(len(distr1)):
        diff = abs(distr1[i] - distr2[i])
        if diff > thres:
            return False
    return True

def findBalance(data, train_percent, thres):
    tifs = len(data)
    class_num = len(data[0])
    for i in range(1000000):
        tr_set, te_set = train_test_split(np.array(list(range(len(data)))),\
                                          test_size=1-train_percent)
        tr_d = showDistribution(data, tr_set)
        te_d = showDistribution(data, te_set)
        check = checkThreshold(tr_d, te_d, thres)
        if (check == -1):
            return -1
        elif (check == True):
            return tr_set, te_set
    return [], []

In [180]:
def splitTrainValidTest(all_chips,all_classes):
    # make the table where filename is rowwise, columns are class
    s_chips = all_chips
    s_classes = all_classes
    # Count number of classes
    results = transpose_class_counts(s_chips,s_classes)
    train_ind, test_ind = findBalance(results[:,1:], 0.7, 0.02)
    test_tifs = indToTifName(results, test_ind)
    
    # Split training set into train and validation
    train_tifs = indToTifName(results, train_ind)
    train_mask = np.isin(s_chips,train_tifs)
    
    # Find split for training and validation
    train_chips = s_chips[train_mask]
    train_classes = s_classes[train_mask]
    
    train_results = transpose_class_counts(train_chips,train_classes)
    train_ind, valid_ind = findBalance(train_results[:,1:], 0.7, 0.02)
    # export new train and valid tif labels
    train_tifs = indToTifName(train_results, train_ind)
    valid_tifs = indToTifName(train_results, valid_ind)
    return (train_tifs,valid_tifs, test_tifs)

idxs = range(0,40000)
string_sets = ["train","valid","test"]
data_sets = splitTrainValidTest(chips1[idxs],classes1[idxs])

20it [00:00, 1612.02it/s]
13it [00:00, 2882.23it/s]


In [181]:
for idx,str_set in enumerate(string_sets):
    with open("{}_tifs.pkl".format(str_set),"wb") as f:
        pickle.dump(data_sets[idx],f)
        pass
    pass

In [174]:
print(np.nonzero(np.isin(data_sets[0],data_sets[1])))
print(np.nonzero(np.isin(data_sets[0],data_sets[2])))
print(np.nonzero(np.isin(data_sets[1],data_sets[2])))

(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
